In [2]:
import numpy as np 
import json
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

In [20]:
CHATBOT_FILE = 'E:\BANGKIT2022\CapstoneProject\data_gathering\data_chatbot\intents.json'

# Read the data
f = open(CHATBOT_FILE)
load = json.load(f)

X_train = []
y_train = []
for i in load['intents']:
    for x in i['patterns']:
        X_train.append(x.lower())
        y_train.append(i['tag'])



In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
total_words = len(tokenizer.word_index) + 1

In [22]:
print(tokenizer.word_index)

{'selamat': 1, 'makanan': 2, 'halo': 3, 'pagi': 4, 'siang': 5, 'malam': 6, 'tolong': 7, 'rekomendasikan': 8, 'rekomendasi': 9, 'meal': 10, 'yang': 11, 'cocok': 12, 'saran': 13}


In [23]:
tokenizer.texts_to_sequences(X_train)

[[3],
 [1, 5],
 [1, 4],
 [1, 6],
 [3, 1, 4],
 [7, 8, 2],
 [9, 10],
 [2, 11, 12],
 [13, 2]]

In [24]:
def n_gram_seqs(corpus, tokenizer, label):
	input_sequences = []
	new_label = []
	### START CODE HERE
	for index, line in enumerate(corpus):
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(0, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)
			new_label.append(label[index])
	### END CODE HERE
	 
	return input_sequences, new_label

In [25]:
input_sequences, y_train = n_gram_seqs(X_train, tokenizer, y_train)

max_sequence_len = max([len(x) for x in input_sequences])

print(f"n_grams of input_sequences have length: {len(input_sequences)}")
print(f"maximum length of sequences is: {max_sequence_len}")

n_grams of input_sequences have length: 20
maximum length of sequences is: 3


In [26]:
# GRADED FUNCTION: pad_seqs
def pad_seqs(input_sequences, maxlen):
    ### START CODE HERE
    padded_sequences = pad_sequences(input_sequences, maxlen=maxlen)
    
    return padded_sequences
    ### END CODE HERE

In [27]:
input_sequences = pad_seqs(input_sequences, max_sequence_len)

print(f"padded corpus has shape: {input_sequences.shape}")
print(input_sequences, y_train)

padded corpus has shape: (20, 3)
[[ 0  0  3]
 [ 0  0  1]
 [ 0  1  5]
 [ 0  0  1]
 [ 0  1  4]
 [ 0  0  1]
 [ 0  1  6]
 [ 0  0  3]
 [ 0  3  1]
 [ 3  1  4]
 [ 0  0  7]
 [ 0  7  8]
 [ 7  8  2]
 [ 0  0  9]
 [ 0  9 10]
 [ 0  0  2]
 [ 0  2 11]
 [ 2 11 12]
 [ 0  0 13]
 [ 0 13  2]] ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi']


In [28]:
preprocessing.LabelEncoder()
label_encoder = preprocessing.LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [29]:
y_train_encoded

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [30]:
model = tf.keras.Sequential([
  layers.Input(shape=(3)),
  layers.Dense(32, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(len(y_train_encoded), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=input_sequences,
          y=y_train_encoded,
          epochs=500)

Epoch 1/500
1/1 [==============================] - 0s 266ms/step - loss: 4.5472 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 6ms/step - loss: 3.1191 - accuracy: 0.0000e+00
Epoch 3/500
1/1 [==============================] - 0s 7ms/step - loss: 2.0947 - accuracy: 0.3000
Epoch 4/500
1/1 [==============================] - 0s 7ms/step - loss: 1.5862 - accuracy: 0.5000
Epoch 5/500
1/1 [==============================] - 0s 6ms/step - loss: 1.3504 - accuracy: 0.5000
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 1.1800 - accuracy: 0.5000
Epoch 7/500
1/1 [==============================] - 0s 7ms/step - loss: 1.0242 - accuracy: 0.6500
Epoch 8/500
1/1 [==============================] - 0s 8ms/step - loss: 0.8871 - accuracy: 0.6000
Epoch 9/500
1/1 [==============================] - 0s 9ms/step - loss: 0.8128 - accuracy: 0.7000
Epoch 10/500
1/1 [==============================] - 0s 7ms/step - loss: 0.7934 - accuracy: 0.7000
Epoch 11/500
1/1 [=

In [37]:
seed_text = 'halo, selamat siang'

token_list = tokenizer.texts_to_sequences([seed_text])[0]
print(token_list)
	# Pad the sequences
token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')

predicted = model.predict(token_list, verbose=0)
	# Choose the next word based on the maximum probability
predicted = np.argmax(predicted, axis=-1).item()

[3, 1, 5]


In [39]:
label = ['greeting','rekomendasi']
print(predicted, label[predicted])

0 greeting


In [19]:
saved_model_path = "./saved_model/simple_chatbot_model/first_model.h5"
tokenizer_path = "./saved_model/simple_chatbot_model/tokenizer.json"
max_seq_path = "./saved_model/simple_chatbot_model/max_sequence_length.txt"
# YOUR CODE HERE
model.save(saved_model_path)
with open(tokenizer_path, 'w') as file:
    json.dump(tokenizer.to_json(), file)
    file.close()
with open(max_seq_path, 'w') as file:
    file.write(str(max_sequence_len))
    file.close()